请点击[此处](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576)查看本环境基本用法.  <br>
Please click [here ](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576) for more detailed instructions. 

# 基于文心大模型和PaddleSpeech控制无人机飞行！

## 0 项目介绍
主要参考论文《[ChatGPT for Robotics: Design Principles and Model Abilities](https://www.microsoft.com/en-us/research/group/autonomous-systems-group-robotics/articles/chatgpt-for-robotics/)》开发。结合文心大模型ernie，实现无人机的基本控制和任务执行。主要展示如何利用提示词工程，输出操作无人机的代码，进而实现从"工程师"在环，到"用户"在环，以大幅降低无人机使用门槛。


我们按照目前agent较为通用的框架来编写本项目内容。一个基于LLM的自主Agent系统一般包括下面几个关键部分：
1. Planning（规划）。一般是指将大型任务分解为更小、更易管理的子目标，从而有效地处理复杂任务。
2. Memory（记忆）,也就是知识库。如短期记忆，就是prompt或者对话上下文中的信息。长期记忆，典型如langchain向量知识库。
3. Tool use（工具使用）。通过输出指令，来调用额外的API，如进行信息获取、代码执行等。

## 1 arisim仿真系统基本概念
AirSim是一款基于UE游戏引擎的开源跨平台仿真器，可以用于无人机、无人车等机器人的物理和视觉仿真。

具体到无人机操作，经常采用“offboard”模式，也就是不用管具体的无人机加速、减速、翻滚等操作，只需要指定目标点，让无人机飞过去即可。

唯一需要知道的是坐标系的定义。Airsim的坐标系，采用一种叫做NED(北东地)的坐标系，也就是+X是北，+Y是东，+Z是下，注意地面朝上，是Z轴负方向。

无人机初始位置记为[0,0,0]，面向正北方，也就是原点位置位于地图左下角。

本项目的场景大致如下所示：


<img src="https://ai-studio-static-online.cdn.bcebos.com/47e5b83f51e741188f6e65e4e0b97db056833e7c9d4e4acdaa55edb26fda2248" width="600" height="600" />





## 2 PaddleSpeech介绍和安装

飞桨语音模型库PaddleSpeech，为开发者提供了语音识别、语音合成、声纹识别、声音分类等多种语音交互能力，代码全部开源，各类服务一键部署，并附带保姆级教学文档，让开发者轻松搞定产业级应用！

In [1]:
# 安装PaddleSpeech
# !git clone https://gitee.com/paddlepaddle/PaddleSpeech.git
# 进入PaddleSpeech目录
%cd /home/aistudio/PaddleSpeech
# 安装依赖环境
!pip install pytest-runner --user
# 需要本地安装
# !pip install openvino==2022.1.0
!pip install . --user

/home/aistudio/PaddleSpeech


/opt/conda/envs/python35-paddle120-env/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


Looking in indexes: https://mirror.baidu.com/pypi/simple/, https://mirrors.aliyun.com/pypi/simple/, https://pypi.tuna.tsinghua.edu.cn/simple/
Looking in indexes: https://mirror.baidu.com/pypi/simple/, https://mirrors.aliyun.com/pypi/simple/, https://pypi.tuna.tsinghua.edu.cn/simple/
Processing /home/aistudio/PaddleSpeech
  Preparing metadata (setup.py) ... done
  Created wheel for paddlespeech: filename=paddlespeech-0.0.0-py3-none-any.whl size=1611805 sha256=cdd4f1b299d4025a686e7ca9f046f4b4342accf0db909b439b383071c44e052a
  Stored in directory: /tmp/pip-ephem-wheel-cache-i1y5024u/wheels/be/ea/03/c6a38fac622d550de807cf091e74a164be0b20aab2b2c1b6ad
Successfully built paddlespeech
  Attempting uninstall: paddlespeech
    Found existing installation: paddlespeech 0.0.0
    Uninstalling paddlespeech-0.0.0:
      Successfully uninstalled paddlespeech-0.0.0
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


## 3 背景知识库-system memory

背景知识设置类似于cosplay，让ernie现在处在一个无人机工程师上班的状态，而不是在玩王者荣耀。

airsm的system知识描述如下所述：

In [1]:
sysprompt="""
你是一名助手，正在帮助我使用AirSim无人机模拟器。
当我要求你做某事时，你应该给我使用AirSim完成任务所需的Python代码，然后解释该代码的作用。
您只能使用我为您定义的函数。
你不能使用任何其他你认为可能存在的假设函数。
您可以使用math和numpy等库中的简单Python函数。
"""

## 4 记忆知识库-short memory 
记忆知识库，就是前面讲的内容，我们通过多轮历史对话的形式，教给ernie

如上所述，这类软硬件结合的项目，prompt好的标准就是，**一个初中级python水平的码农，看了之后能完成无人机基本操作任务和任务。**

ernie学了应该也行：）

项目的记忆知识库如下所示：

In [2]:
knowledge_prompt="""
以下是您可以用来命令无人机的一些功能。

aw.takeoff() - 起飞无人机。
aw.land() - 无人机着陆。
aw.get_drone_position() - 以与 XYZ 坐标相对应的 3 个浮点数的列表形式返回无人机的当前位置。
aw.fly_to([x, y, z]) - 将无人机飞到指定的位置，该位置由对应于 X、Y、Z 坐标的三个参数列表指定。这里X,Y,Z是世界坐标系的坐标。下同
aw.fly_path(points) - 沿着点列表指定的路径飞行无人机。每个点又是一个由 3 个浮点数组成的列表，对应于 X、Y、Z 坐标。
aw.set_yaw(yaw) - 将无人机的偏航设置为指定值（以度为单位）。
aw.get_yaw() - 返回无人机当前的偏航角（以度为单位）。
aw.get_position(object_name)：采用字符串作为输入，指示感兴趣对象的名称，并返回 3 个浮点数的列表，指示其 X、Y、Z 坐标。

一些有用的东西：
您应该使用我为您定义的函数fly_to()，而不是moveToPositionAsync() 或moveToZAsync()。
如果您对某件事不确定，您可以向我提出一个澄清问题，只要您明确指出“问题”即可。
以下是一个示例场景，说明如何提出澄清问题。让我们假设一个场景包含两个球体。

我：飞向球体。
你：问题——有两个球体。你想让我飞向哪一个？
我：请点 1 号球。

场景中存在以下对象，您将使用这些确切的名称来引用它们：

风力发电机1、风力发电机2、太阳能电池板、汽车、人群、塔1、塔2、塔3。

请注意，在函数调用的时候，请使用对应的英文名称，具体对应如下：
turbine1 ：风力发电机1
turbine2 ：风力发电机2
solarpanels：太阳能电池板
car：汽车
crowd:人群
tower1:塔1
tower2:塔2
tower3:塔3

例如，你需要获得风力发电机1的位置，你可以这样写：
aw.get_position("turbine1")
而不是：
aw.get_position("风力发电机1")

除了无人机本身之外，所有物体都不可移动。请记住，有两个涡轮机和三个塔。当有多个相同类型的对象时，
如果我没有明确指定我指的是哪个对象，您应该始终要求我进行澄清。永远不要做出假设。

就坐标轴约定而言，我们使用的是世界坐标系NED，也就是"北动地"坐标系，
，即+X是北，+Y是东，+Z是下。这意味着Z值越高越负，如果原点在地面上，z值是零，地面以上是负的！

但人类操作无人机时，往往习惯以机体坐标系"前右下"发送指令。
无人机初始位置为[0,0,0]，面向北方，向前/北表示 X 轴正方向。右/东表示 Y 轴正方向。向下/地表示 Z 轴负方向。

需要注意，每次执行移动任务时候，都需要先获得无人机当前位置的世界坐标系[X, Y, Z]为基准，进行位置的操作加减。

因此，人发出的基于"前右下"的指令，需要和世界坐标系的X,Y,Z进行转换。
具体的转换关系如下：

如向上飞，则是Z轴当前值减去一个正数数值，

而左右方向，和当前无人机朝向有关，
如无人机朝向北方，向右飞，Y轴增加一个正数数值。向前飞，X轴增加一个正数数值，向右下方移动，则Y轴增加一个正数数值，Z则是轴减去一个正数数值。

如无人机朝向东方，向右飞，X轴减去一个正数数值。向前飞，Y轴增加一个正数数值。

我们的实验只会考虑上述两个朝向，而不考虑更复杂的转换情况。
"""

## 5 推理和对话

首先，我们接入ernie的sdk

In [4]:
# 安装erniebot
!pip install erniebot

Looking in indexes: https://mirror.baidu.com/pypi/simple/, https://mirrors.aliyun.com/pypi/simple/, https://pypi.tuna.tsinghua.edu.cn/simple/


In [3]:
import erniebot
import re

#api key
erniebot.api_type = "aistudio"
erniebot.access_token = "85e4da127a707245f3d26ec617cc4d982bcc5a6c" #添加自己的
MY_MODEL = "ernie-bot-4" #ernie-bot",ernie-bot-turbo,ernie-bot-4,ernie-bot-8k,ernie-text-embedding,ernie-vilg-v2

然后，我们利用历史对话做知识库，也就是把前面的知识都当做历史记录，每次都把全部历史回话发送给erniebot。

这里使用全局的chat_history列表记录回话，并定义了一个ask函数实现知识库功能。

In [4]:
#历史对话话记录，全局变量
chat_history = []

# 调用ernie的chat api，包含历史记录，多轮对话
def ask(prompt):
    # 加入用户输入的prompt
    chat_history.append(
        {
            "role": "user",
            "content": prompt,
        }
    )
    completion = erniebot.ChatCompletion.create(
        model=MY_MODEL,
        messages=chat_history,
        system=sysprompt,  # 前面声明的系统知识
        # temperature=0.000001,
        top_p=0,
    )


    # 加入机器回复，相当于保存全部的历史记录，多轮对话
    chat_history.append(
        {
            "role": "assistant",
            "content": completion.get_result(),
        }
    )

    return chat_history[-1]["content"]

然后我们就可以让ernie完成具体的飞行任务了，

我们首先加入知识库：

In [5]:
ask(knowledge_prompt)

'好的，我理解了您的要求。现在，请告诉我您希望无人机执行的具体任务，我会为您提供相应的Python代码并解释其作用。'

针对不同的应用场景，我们一般还会给出一个回复的模式，这里直接按照ernie的对话格式构造一个例子即可。

因为是直接构造的多轮对话, 直接添加到chat_history即可，无需再次调用ask函数。

In [6]:
example_msg = [
    {
        "role": "user",
        "content": "向上移动 10 个单位"
    },
    {
        "role": "assistant",
        "content": """```python
aw.fly_to([aw.get_drone_position()[0], aw.get_drone_position()[1], aw.get_drone_position()[2]-10])
```

此代码使用“fly_to()”函数将无人机移动到距当前位置 10 个单位的新位置。它通过使用 get_drone_position() 获取无人机的当前位置，然后创建一个具有相同 X 和 Y 坐标但 Z 坐标减去 10 的新列表来实现这一点。然后无人机将使用以下命令飞到这个新位置： `fly_to()`。 """}
]

chat_history.extend(example_msg)

我们希望能能通过python的eval函数，直接执行ernie回复中的代码部分，

因此，这里还需要写一个简单的文本解析函数：

In [7]:
def extract_python_code(content):
    """
    Extracts the python code from a response.
    :param content:
    :return:
    """
    code_block_regex = re.compile(r"```(.*?)```", re.DOTALL)
    code_blocks = code_block_regex.findall(content)
    if code_blocks:
        full_code = "\n".join(code_blocks)

        if full_code.startswith("python"):
            full_code = full_code[7:]

        return full_code
    else:
        return None

至此，准备工作全部完成，我们可以用ernie完成飞行任务了。本项目大致完成3个飞行任务：

1. 无人机基本操作。
2. 风力发电机叶片检查。
3. 太阳能电池板阵列巡检。

任务难度依次增加。

考虑到云端执行仿真任务不太方便，我们首先用ernie全部的生成无人机操作代码，然后再在本地电脑的airsim模拟器上执行验证即可。

为了调用更方便，我们再构造一个简单的process函数，输入任务信息，输出ernie生成的python代码。

In [8]:
def process(command):
    #step 1, ask ernie
    response = ask(command)
    print("response=",response)

    #step 2, extract python code
    python_code = extract_python_code(response)
    print("process python_code",python_code)

    #step 3, exec python code, 测试的时候不需要执行
    exec(python_code)
    return python_code

### 任务1：无人机基本操作
让无人机随便飞一飞。

In [9]:
from airsim_wrapper import *
aw = AirSimWrapper() #初始化airsim，airsim基本功能调用

Connected!
Client Ver:1 (Min Req: 1), Server Ver:1 (Min Req: 1)


In [11]:
# 进入data目录
%cd /home/aistudio/data
# 音频预览
import IPython.display as ipd
ipd.Audio("1.wav")

/home/aistudio/data


/opt/conda/envs/python35-paddle120-env/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [43]:
aw.reset()

In [10]:
# from paddlespeech.cli.asr.infer import ASRExecutor
# asr = ASRExecutor()
# %cd /home/aistudio/data
# command = asr(audio_file="1.wav",force_yes=True)
# print("command: ",command)



command="起飞"
python_code = process(command)
print("python_code:\n", python_code)

response= ```python
aw.takeoff()
```

此代码使用“takeoff()”函数使无人机起飞。无人机将从其当前位置起飞并悬停在空中，等待进一步的指令。
process python_code aw.takeoff()
python_code:
 aw.takeoff()


In [13]:
# 进入data目录
%cd /home/aistudio/data
# 音频预览
import IPython.display as ipd
ipd.Audio("2.wav")

/home/aistudio/data


/opt/conda/envs/python35-paddle120-env/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [ ]:
# %cd /home/aistudio/data
# command = asr(audio_file="2.wav",force_yes=True)
# print("command: ",command)
command = "飞到汽车上方"
python_code = process(command)
print("python_code:\n", python_code)

In [12]:
command = "汽车上方应该是Z轴减去一个正数"
python_code = process(command)
print("python_code:\n", python_code)

response= 对不起，我之前的回答有误。您是正确的，为了飞到汽车上方，我们应该在Z轴上减去一个正数。以下是修正后的代码：

```python
car_position = aw.get_position("car")
aw.fly_to([car_position[0], car_position[1], car_position[2]-5])
```

此代码首先获取汽车的当前位置。然后，它创建一个新的位置列表，该列表具有与汽车相同的 X 和 Y 坐标，但 Z 坐标减少了 5 个单位，以确保无人机在汽车上方悬停。最后，无人机使用“fly_to()”函数飞到这个新位置。
process python_code car_position = aw.get_position("car")
aw.fly_to([car_position[0], car_position[1], car_position[2]-5])
python_code:
 car_position = aw.get_position("car")
aw.fly_to([car_position[0], car_position[1], car_position[2]-5])


### 任务2：风力发电机叶片检查
飞到发电机上方，依次检测3个叶片

考虑到几个任务可能互相有干扰，执行新任务之前，我们需要把历史记录重置为初始状态。

In [12]:
#对话记录恢复初始状态
chat_history = chat_history[:4]

In [14]:
aw.reset()

In [18]:
command = "请起飞吧"
python_code = process(command)
print("python_code:", python_code)

response= 好的，以下是起飞无人机的Python代码：


```python
aw.takeoff()
```
这段代码将启动无人机并使其起飞。无人机将从地面起飞并悬停在空中，等待进一步的指令。

请注意，在起飞之前，请确保无人机已经正确设置和校准，并且处于安全的环境中，以避免任何潜在的风险或问题。
process python_code aw.takeoff()
python_code: aw.takeoff()


In [13]:
command = "请选风力发电机1,飞向风力发电机,沿X轴方向，在风力发电机10米之前停下来"
python_code = process(command)
print("python_code:", python_code)

response= ```python
# 获取风力发电机1的位置
turbine1_position = aw.get_position("turbine1")

# 计算无人机需要飞到的位置，即在风力发电机1的X轴方向前10米的位置
target_position = [turbine1_position[0] - 10, turbine1_position[1], turbine1_position[2]]

# 无人机飞向目标位置
aw.fly_to(target_position)
```

这段代码首先使用 `get_position()` 函数获取风力发电机1的位置。然后，它创建一个新的目标位置列表，该列表在X轴方向上比风力发电机1的位置少10米（因为我们希望无人机停在风力发电机之前），而Y轴和Z轴的坐标保持不变。最后，它使用 `fly_to()` 函数命令无人机飞向这个新计算出的目标位置。
process python_code # 获取风力发电机1的位置
turbine1_position = aw.get_position("turbine1")

# 计算无人机需要飞到的位置，即在风力发电机1的X轴方向前10米的位置
target_position = [turbine1_position[0] - 10, turbine1_position[1], turbine1_position[2]]

# 无人机飞向目标位置
aw.fly_to(target_position)
python_code: # 获取风力发电机1的位置
turbine1_position = aw.get_position("turbine1")

# 计算无人机需要飞到的位置，即在风力发电机1的X轴方向前10米的位置
target_position = [turbine1_position[0] - 10, turbine1_position[1], turbine1_position[2]]

# 无人机飞向目标位置
aw.fly_to(target_position)


In [14]:
command = "请使用2号风力发电机。沿X轴与风力发电机2保持12米距离飞向风力发电机，并达到50米的高度的位置。"
python_code = process(command)
print("python_code:", python_code)

response= ```python
# 获取风力发电机2的位置
turbine2_position = aw.get_position("turbine2")

# 计算无人机需要飞到的位置，即在风力发电机2的X轴方向保持12米距离，并达到50米高度的位置
target_position = [turbine2_position[0] - 12, turbine2_position[1], 50]

# 无人机飞向目标位置
aw.fly_to(target_position)
```

这段代码首先使用 `get_position()` 函数获取风力发电机2的位置。然后，它创建一个新的目标位置列表，该列表在X轴方向上比风力发电机2的位置少12米（因为我们希望无人机在风力发电机之前保持12米的距离），Y轴的坐标保持不变，而Z轴的高度设置为50米。最后，它使用 `fly_to()` 函数命令无人机飞向这个新计算出的目标位置。
process python_code # 获取风力发电机2的位置
turbine2_position = aw.get_position("turbine2")

# 计算无人机需要飞到的位置，即在风力发电机2的X轴方向保持12米距离，并达到50米高度的位置
target_position = [turbine2_position[0] - 12, turbine2_position[1], 50]

# 无人机飞向目标位置
aw.fly_to(target_position)
python_code: # 获取风力发电机2的位置
turbine2_position = aw.get_position("turbine2")

# 计算无人机需要飞到的位置，即在风力发电机2的X轴方向保持12米距离，并达到50米高度的位置
target_position = [turbine2_position[0] - 12, turbine2_position[1], 50]

# 无人机飞向目标位置
aw.fly_to(target_position)


In [16]:
command = "请注意，Z坐标是正方向是指向地心的，负方向指向天空，也就是高度应该是-50，请对刚才的回复予以更正"
python_code = process(command)
print("python_code:", python_code)

response= 非常抱歉，您是对的，我之前的描述有误。在NED坐标系中，Z轴正方向确实是指向地心，而负方向指向天空。因此，当提到高度时，我们应该使用负值。对于之前提到的任务，即检查风力发电机的叶片并飞到当前位置上方30米然后再次向下，代码应该更正如下：

```python
# 获取无人机当前位置
current_position = aw.get_drone_position()

# 计算无人机需要飞到的位置，即在当前位置的上方30米（注意Z坐标是负数）
up_position = [current_position[0], current_position[1], current_position[2] - 30]

# 无人机飞向上方30米的位置
aw.fly_to(up_position)

# 计算无人机需要飞到的位置，即再次回到原始位置
down_position = current_position

# 无人机飞回原始位置
aw.fly_to(down_position)
```

这次，当计算向上飞行30米的目标位置时，我们从当前位置的Z坐标中减去30（因为Z轴负方向指向天空），从而得到正确的目标位置。再次感谢您的指正。
process python_code # 获取无人机当前位置
current_position = aw.get_drone_position()

# 计算无人机需要飞到的位置，即在当前位置的上方30米（注意Z坐标是负数）
up_position = [current_position[0], current_position[1], current_position[2] - 30]

# 无人机飞向上方30米的位置
aw.fly_to(up_position)

# 计算无人机需要飞到的位置，即再次回到原始位置
down_position = current_position

# 无人机飞回原始位置
aw.fly_to(down_position)
python_code: # 获取无人机当前位置
current_position = aw.get_drone_position()

# 计算无人机需要飞到的位置，即在当前位置的上方30米（注意Z坐标是负数）
up_position = [current_posit

In [17]:
command = "现在让我们开始检查风力发电机的叶片。第一个叶片垂直向上，请飞到当前位置上方 30 米，然后再次向下。"
python_code = process(command)
print("python_code:", python_code)

response= ```python
# 获取无人机当前位置
current_position = aw.get_drone_position()

# 计算无人机需要飞到的位置，即在当前位置的上方30米
up_position = [current_position[0], current_position[1], current_position[2] - 30]

# 无人机飞向上方30米的位置
aw.fly_to(up_position)

# 无人机进行叶片检查操作...

# 计算无人机需要飞到的位置，即再次回到原始位置
down_position = current_position

# 无人机飞回原始位置
aw.fly_to(down_position)
```

这段代码首先获取无人机的当前位置，并存储在一个变量中。然后，它计算无人机需要飞到的目标位置，即在当前位置的上方30米（通过从当前位置的Z坐标中减去30来实现）。接下来，它使用`fly_to()`函数命令无人机飞向这个目标位置。在无人机到达目标位置后，可以进行叶片检查操作（这里假设有相应的代码来处理叶片检查任务）。最后，它再次使用`fly_to()`函数命令无人机飞回原始位置，即之前存储的当前位置。
process python_code # 获取无人机当前位置
current_position = aw.get_drone_position()

# 计算无人机需要飞到的位置，即在当前位置的上方30米
up_position = [current_position[0], current_position[1], current_position[2] - 30]

# 无人机飞向上方30米的位置
aw.fly_to(up_position)

# 无人机进行叶片检查操作...

# 计算无人机需要飞到的位置，即再次回到原始位置
down_position = current_position

# 无人机飞回原始位置
aw.fly_to(down_position)
python_code: # 获取无人机当前位置
current_position = aw.get_drone_position()

# 计算无人机需要飞到的位置，即在当前位置的上方30米
up_position =

In [18]:
command = "为了检查第二个叶片，我需要你以与水平轴Y成-30度角的右下方向飞行。您应该在YZ 平面上飞行30 米的距离，然后返回当前位置。"
python_code = process(command)
print("python_code:", python_code)

response= ```python
# 获取无人机当前位置
current_position = aw.get_drone_position()

# 计算无人机需要飞到的位置，即以与水平轴Y成-30度角的右下方向飞行30米
import math
angle = -30  # 角度，负值表示逆时针旋转
distance = 30  # 飞行距离
dx = distance * math.cos(math.radians(angle))  # 在X轴上的位移
dz = distance * math.sin(math.radians(angle))  # 在Z轴上的位移
target_position = [current_position[0] + dx, current_position[1], current_position[2] - dz]

# 无人机飞向目标位置
aw.fly_to(target_position)

# 进行叶片检查操作...

# 无人机飞回原始位置
aw.fly_to(current_position)
```

这段代码首先获取无人机的当前位置，并存储在一个变量中。然后，它使用三角函数计算无人机需要飞到的目标位置。这里，我们假设无人机在YZ平面上飞行，与水平轴Y成-30度角，并且飞行距离为30米。通过计算X轴和Z轴上的位移，我们可以得到目标位置的坐标。接下来，它使用`fly_to()`函数命令无人机飞向这个目标位置。在无人机到达目标位置后，可以进行叶片检查操作（这里假设有相应的代码来处理叶片检查任务）。最后，它再次使用`fly_to()`函数命令无人机飞回原始位置，即之前存储的当前位置。
process python_code # 获取无人机当前位置
current_position = aw.get_drone_position()

# 计算无人机需要飞到的位置，即以与水平轴Y成-30度角的右下方向飞行30米
import math
angle = -30  # 角度，负值表示逆时针旋转
distance = 30  # 飞行距离
dx = distance * math.cos(math.radians(angle))  # 在X轴上的位移
dz = distance * math.sin(math.radians(a

KeyboardInterrupt: 

In [24]:
command = "有个小问题，向右下方移动，Z轴应该是减去一个负数，或者加上一个正数，而-30度的sin值是负数,也就是z轴减去z_offset"
python_code = process(command)
print("python_code:", python_code)

response= 非常抱歉，您是对的。我之前的回复中确实存在错误。感谢您的指正。以下是更正后的代码：

```python
# 获取无人机当前位置
current_position = aw.get_drone_position()

# 计算无人机需要飞到的位置，即在YZ平面上以-30度角飞行30米的距离
y_offset = 30 * math.cos(math.radians(-30))
z_offset = 30 * math.sin(math.radians(-30))
target_position = [current_position[0], current_position[1] - y_offset, current_position[2] - z_offset]

# 无人机飞向目标位置
aw.fly_to(target_position)

# 无人机返回当前位置
aw.fly_to(current_position)
```

在这段更正后的代码中，我修改了目标位置的计算，将Z轴的偏移量改为减去一个负数，或者加上一个正数，以符合您的指正。无人机将飞向目标位置进行检查，并返回原始位置。再次感谢您的指正，并对此前的错误表示歉意。
process python_code # 获取无人机当前位置
current_position = aw.get_drone_position()

# 计算无人机需要飞到的位置，即在YZ平面上以-30度角飞行30米的距离
y_offset = 30 * math.cos(math.radians(-30))
z_offset = 30 * math.sin(math.radians(-30))
target_position = [current_position[0], current_position[1] - y_offset, current_position[2] - z_offset]

# 无人机飞向目标位置
aw.fly_to(target_position)

# 无人机返回当前位置
aw.fly_to(current_position)
python_code: # 获取无人机当前位置
current_position = aw.get_drone_position()

# 计算无人机需要飞

In [25]:
command = "还是有问题，z轴应该是加上一个正数"
python_code = process(command)
print("python_code:", python_code)

response= 非常抱歉，我之前的回复中仍然存在错误。感谢您的耐心指正。以下是再次更正后的代码：

```python
# 获取无人机当前位置
current_position = aw.get_drone_position()

# 计算无人机需要飞到的位置，即在YZ平面上以-30度角飞行30米的距离
y_offset = 30 * math.cos(math.radians(-30))
z_offset = abs(30 * math.sin(math.radians(-30)))
target_position = [current_position[0], current_position[1] - y_offset, current_position[2] + z_offset]

# 无人机飞向目标位置
aw.fly_to(target_position)

# 无人机返回当前位置
aw.fly_to(current_position)
```

在这段再次更正后的代码中，我修改了Z轴偏移量的计算，将其改为加上一个正数，以确保无人机在YZ平面上以-30度角飞行30米的距离。无人机将飞向目标位置进行检查，并返回原始位置。再次感谢您的指正，并对此前的错误表示歉意。
process python_code # 获取无人机当前位置
current_position = aw.get_drone_position()

# 计算无人机需要飞到的位置，即在YZ平面上以-30度角飞行30米的距离
y_offset = 30 * math.cos(math.radians(-30))
z_offset = abs(30 * math.sin(math.radians(-30)))
target_position = [current_position[0], current_position[1] - y_offset, current_position[2] + z_offset]

# 无人机飞向目标位置
aw.fly_to(target_position)

# 无人机返回当前位置
aw.fly_to(current_position)
python_code: # 获取无人机当前位置
current_position = aw.get_drone_posit

In [29]:
command = "现在我们还需要检查叶片的背面。我需要你重复刚才所做的事情，但到了最后就停下来"
python_code = process(command)
print("python_code:", python_code)

response= ```python
# 获取无人机当前位置
current_position = aw.get_drone_position()

# 计算无人机需要飞到的位置，即在YZ平面上以-30度角飞行30米的距离
y_offset = 30 * math.cos(math.radians(-30))
z_offset = abs(30 * math.sin(math.radians(-30)))
target_position = [current_position[0], current_position[1] - y_offset, current_position[2] + z_offset]

# 无人机飞向目标位置
aw.fly_to(target_position)

# 检查叶片的背面
# 在这里执行检查叶片背面的操作

# 无人机停在目标位置，不返回原始位置
# 由于aw对象没有hover方法，无人机将保持在最后的位置而不进行任何额外操作
```

在这段代码中，无人机将飞向目标位置以检查叶片的背面。由于没有`hover`方法，无人机将保持在最后的位置而不进行任何额外操作。您可以在`# 在这里执行检查叶片背面的操作`的注释下添加适当的代码来执行叶片背面的检查。请确保在执行这段代码之前，无人机已经处于适合飞行的状态，并且没有其他障碍物在飞行路径上。
process python_code # 获取无人机当前位置
current_position = aw.get_drone_position()

# 计算无人机需要飞到的位置，即在YZ平面上以-30度角飞行30米的距离
y_offset = 30 * math.cos(math.radians(-30))
z_offset = abs(30 * math.sin(math.radians(-30)))
target_position = [current_position[0], current_position[1] - y_offset, current_position[2] + z_offset]

# 无人机飞向目标位置
aw.fly_to(target_position)

# 检查叶片的背面
# 在这里执行检查叶片背面的操作

# 无人机停在目标位置，不返回原始位置
# 由于

In [28]:
command = "上面一个操作，没有hover方法，请从新生成"
python_code = process(command)
print("python_code:", python_code)

response= 非常抱歉，我之前的回复中提到了不存在的`hover`方法。以下是经过修正的代码：

```python
# 获取无人机当前位置
current_position = aw.get_drone_position()

# 计算无人机需要飞到的位置，即在X轴上飞行20米，再加上几米的间隙
target_position = [current_position[0] + 20 + gap, current_position[1], current_position[2]]

# 无人机飞向目标位置
aw.fly_to(target_position)

# 将无人机旋转180度
aw.set_yaw(aw.get_yaw() + 180)

# 让无人机悬停在当前位置
aw.hover()
```

在这段修正后的代码中，我添加了`aw.hover()`来让无人机悬停在当前位置。这样，无人机将飞向目标位置，旋转180度后，将保持在悬停状态。请确保在执行这段代码之前，无人机已经处于适合飞行的状态，并且没有其他障碍物在飞行路径上。
process python_code # 获取无人机当前位置
current_position = aw.get_drone_position()

# 计算无人机需要飞到的位置，即在X轴上飞行20米，再加上几米的间隙
target_position = [current_position[0] + 20 + gap, current_position[1], current_position[2]]

# 无人机飞向目标位置
aw.fly_to(target_position)

# 将无人机旋转180度
aw.set_yaw(aw.get_yaw() + 180)

# 让无人机悬停在当前位置
aw.hover()


NameError: name 'gap' is not defined

In [27]:
command ="现在向前移动，飞到风力发电机后面。在 X 轴上飞行 20 米，再加上几米的间隙，然后将无人机旋转 180 度。您可以使用 set_yaw() 函数来旋转无人ho机。"
python_code = process(command)
print("python_code:", python_code)

response= ```python
# 获取无人机当前位置
current_position = aw.get_drone_position()

# 计算无人机需要飞到的位置，即在X轴上飞行20米，再加上几米的间隙
target_position = [current_position[0] + 20 + gap, current_position[1], current_position[2]]

# 无人机飞向目标位置
aw.fly_to(target_position)

# 将无人机旋转180度
aw.set_yaw(aw.get_yaw() + 180)
```

在这段代码中，首先获取无人机当前的位置，然后计算无人机需要飞到的位置，即在X轴上飞行20米，再加上几米的间隙。无人机将飞向这个目标位置。完成移动后，使用`set_yaw()`函数将无人机旋转180度，以便无人机面向风力发电机的背面。请确保在执行这段代码之前，无人机已经处于适合飞行的状态，并且没有其他障碍物在飞行路径上。
process python_code # 获取无人机当前位置
current_position = aw.get_drone_position()

# 计算无人机需要飞到的位置，即在X轴上飞行20米，再加上几米的间隙
target_position = [current_position[0] + 20 + gap, current_position[1], current_position[2]]

# 无人机飞向目标位置
aw.fly_to(target_position)

# 将无人机旋转180度
aw.set_yaw(aw.get_yaw() + 180)


NameError: name 'gap' is not defined

In [26]:
command = "太好了！现在我要你检查叶片的背面。这意味着你所做的与之前所做的完全相反，也就是说，朝右上角的方向飞行。"
python_code = process(command)
print("python_code:", python_code)

python_code: import math

# 获取无人机当前位置
current_position = aw.get_drone_position()

# 计算飞行距离在Y和Z轴上的分量
distance_y = 30 * math.cos(math.radians(30))
distance_z = 30 * math.sin(math.radians(30))

# 计算目标位置的Y和Z坐标
target_y = current_position[1] - distance_y
target_z = current_position[2] + distance_z

# 保持X坐标不变
target_x = current_position[0]

# 沿与水平轴Y成30度角的右上方向飞行到目标位置
aw.fly_to([target_x, target_y, target_z])



In [27]:
command = "还是有个小问题，你现在是到了叶片背面，并且旋转了180度，以你的视角来看，是和之前左右和前后反转了。向右上方移动，Y轴应该是减去一个正数。而高度是增加了，因此Z轴应该是减去一个正数"
python_code = process(command)
print("python_code:", python_code)

python_code: import math

# 获取无人机当前位置
current_position = aw.get_drone_position()

# 计算飞行距离在Y和Z轴上的分量
distance_y = 30 * math.cos(math.radians(30))
distance_z = 30 * math.sin(math.radians(30))

# 计算目标位置的Y和Z坐标
target_y = current_position[1] - abs(distance_y) # 使用绝对值确保Y轴方向正确，并减去正数
target_z = current_position[2] - abs(distance_z) # 使用绝对值确保Z轴方向正确，并减去正数

# 保持X坐标不变
target_x = current_position[0]

# 沿与水平轴Y成30度角的右上方向飞行到目标位置
aw.fly_to([target_x, target_y, target_z])



### 任务3：太阳能电池板阵列巡检
这个任务具有一定的挑战性，需要ernie自己构造一个循环，来完成任务。

经过多次尝试，ernie很难一次完成任务，因此，采用分步的方法，顺利完成任务。

嗯，教学生的好方法。

In [28]:
#对话记录恢复初始状态
chat_history = chat_history[:4]

In [29]:
command = "请起飞吧"
python_code = process(command)
print("python_code:", python_code)

python_code: aw.takeoff()



In [30]:
command = "顺时针旋转90度，飞向太阳能电池板。"
python_code = process(command)
print("python_code:", python_code)

APIError: 访问过于频繁，请稍候再试

In [ ]:
command = """
现在无人机朝向东方，处在太阳能电池板阵列第一行的最左边。

太阳能电池板阵列宽30米（和X轴平行），长50米(和Y轴平行)

请始终保持5米的高度飞行。

沿宽度方向，向右飞到第一行的最右端。
注意，此时X坐标应该是减去一个正数。

然后向后飞行6米，也就是沿长度方向，到达第二行的最右端。

之后再向左做飞行，到达第二行的最左侧，

然后再向后飞行6米，到达下一行。
"""
python_code = process(command)
print("python_code:", python_code)

In [ ]:
command = "太棒了，重复上面动作3次"
python_code = process(command)
print("python_code:", python_code)

模拟仿真系统中实际效果可以参考本地版


## 6 小结和规划

无人机大模型应用，能够体现行业应用的一些特点：领域非常垂直，大模型学到的知识相对较少，逻辑思维要求高，还要尽量避免回复的随机性。

因此，非常考验prompt的构造水平。

无人系统还大量涉及各类3维坐标系和转换，如果缺乏空间想象力，人类自己都很难完成任务，这可能也是大语言模型的弱项。

当然，如果具备多模态能力，能够看图识图，效果应该会好很多。

<br/>

文心几个模型的“无人机操作教学评价”：

ernie-bot-turbo：感觉不太聪明的样子。经常不按照格式回复。

ernie-bot：基本可用，普普通通。

ernie-bot-4 ：一学就会，触类旁通，ssp好学生。


<br/>


当然，如果需要进一步的提高业务水平，具体的方法就是大模型的提升路径： 

知识库----> 微调----> 持续预训练。

项目的后续规划，其实也是按照这个路径来：
1. 无人机自主感知。主要是视觉识别，如利用PaddleDetection做观测，给到ernie做感知。这样好比你说“我渴了”，无人机会自动搜索附近的物品，为你找到可乐等物品，这个还是原论文中的内容。
2. 更炫酷。基于paddle的asr/tts等模型，让无人机能和使用者直接语音交互，忽悠PR必备。
3. 完整的无人机大模型。以文心大模型为核心，按照知识库---微调--持续预训练的套路，做一个有点用处的垂直领域开源大模型。

